#### LSTM-base-language-model

- Model:
- Data: 

In [3]:
import tensorflow as tf

import numpy as np

import preprocess_helper
import embedding 

In [4]:
# @TODO define a class Model
class Model():
  
  def __init__(self):
    self.batch_size = 64
    self.epoch = 10 

In [ ]:
# @TODO lines that will be used to define the Model 

# Define the model in Tensorflow ...

sentence_length = 30        # number of words per sentence (ie. how many times we should enroll the network)
batch_size = 64             # number of sentences analysed per batch 
embedding_dimensions = 100  # dimension of the embedding 
state_size = 512            # dimension of the hidden state 
max_grad_norm = 10          # max norm of the gradient 
vocabulary_size = 20000     # vocabulary size 

embedding = np.random.uniform(low=0, high=1, size=embedding_dimensions)

# place holder for a batch dim= 30 x 64 x 100
batch = tf.placeholder(tf.float32, [sentence_length, batch_size, embedding_dimensions])

# var for the embedding of dim=20k x 100
embedding = tf.get_variable("embedding", [vocabulary_size, embedding_dimensions], dtype=tf.float32)

# this i don't know what it is
inputs = tf.nn.embedding_lookup(embedding,
                                input_data)

# construct basic LSTM cell 
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

# Initial state of the LSTM memory.
hidden_state = tf.zeros([batch_size, lstm.state_size])
current_state = tf.zeros([batch_size, lstm.state_size])
state = hidden_state, current_state
probabilities = []
loss = 0.0

# loop over each word in the batch of sentences 
for current_words in batch:
  
    # The value of state is updated after processing each batch of words.
    output, state = lstm(current_words, state)

    # The LSTM output can be used to make next word predictions
    logits = tf.matmul(output, weights) + bias
    probabilities.append(tf.nn.softmax(logits))
    loss += tf.losses.sparse_softmax_cross_entropy(
                labels,
                logits,
                weights=1.0,
                scope=None,
                loss_collection=tf.GraphKeys.LOSSES,
                reduction=Reduction.SUM_BY_NONZERO_WEIGHTS)
    
    cost = tf.reduce_sum(loss)
    
# once the loss was calculated, backprop to update the weights 
learning_rate = tf.Variable(0.0, trainable=False)

tvars = tf.trainable_variables()

grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                  max_grad_norm)

optimizer = tf.train.GradientDescentOptimizer(self._lr)

In [ ]:
# simplified version of the Tensorflow tutorial on RNNs
# available at: https://www.tensorflow.org/tutorials/recurrent

def main():

  frequent_words = load_frequent_words('google-10000-english/20k.txt')
  test_data = load_and_process_data('data/sentences.eval', frequent_words, 28)
  train_data = load_and_process_data('data/sentences.train', frequent_words, 28)

  config = get_config()
  eval_config = get_config()
  eval_config.batch_size = 1
  eval_config.num_steps = 1

  with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)

    with tf.name_scope("Train"):
      train_input = PTBInput(config=config, data=train_data, name="TrainInput")
      with tf.variable_scope("Model", reuse=None, initializer=initializer):
        m = PTBModel(is_training=True, config=config, input_=train_input)
      tf.summary.scalar("Training Loss", m.cost)
      tf.summary.scalar("Learning Rate", m.lr)

    with tf.name_scope("Valid"):
      valid_input = PTBInput(config=config, data=valid_data, name="ValidInput")
      with tf.variable_scope("Model", reuse=True, initializer=initializer):
        mvalid = PTBModel(is_training=False, config=config, input_=valid_input)
      tf.summary.scalar("Validation Loss", mvalid.cost)

    with tf.name_scope("Test"):
      test_input = PTBInput(
          config=eval_config, data=test_data, name="TestInput")
      with tf.variable_scope("Model", reuse=True, initializer=initializer):
        mtest = PTBModel(is_training=False, config=eval_config,
                         input_=test_input)

    models = {"Train": m, "Valid": mvalid, "Test": mtest}
    for name, model in models.items():
      model.export_ops(name)
    metagraph = tf.train.export_meta_graph()

  with tf.Graph().as_default():
    tf.train.import_meta_graph(metagraph)
    for model in models.values():
      model.import_ops()

    for i in range(config.max_max_epoch):
      lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
      m.assign_lr(session, config.learning_rate * lr_decay)

      print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
      train_perplexity = run_epoch(session, m, eval_op=m.train_op,
                                   verbose=True)
      print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
      valid_perplexity = run_epoch(session, mvalid)
      print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

    test_perplexity = run_epoch(session, mtest)
    print("Test Perplexity: %.3f" % test_perplexity)


if __name__ == "__main__":
  tf.app.run()
